In [37]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
drawing_spec = mp_drawing.DrawingSpec(thickness=2, circle_radius=2, color=(255, 0, 255))


# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    A = distance.euclidean(eye_landmarks[1], eye_landmarks[5])
    B = distance.euclidean(eye_landmarks[2], eye_landmarks[4])
    C = distance.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Threshold to determine if eyes are closed
EAR_THRESHOLD = 0.3

cap = cv2.VideoCapture(0)

while True:
    start = time.time()
    success, image = cap.read()
    image = cv2.resize(image, (1200, 800))
    
    # Process image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = face_mesh.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    
    face_3d = []
    face_2d = []
    eyes_closed = False

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    face_2d.append([x, y])
                    face_3d.append([x, y, lm.z])
                    
                    
            # Get right and left eye landmarks
            right_eye_landmarks = [(face_landmarks.landmark[i].x * img_w, face_landmarks.landmark[i].y * img_h) 
                                   for i in [33, 160, 158, 133, 153, 144]]
            left_eye_landmarks = [(face_landmarks.landmark[i].x * img_w, face_landmarks.landmark[i].y * img_h) 
                                  for i in [263, 387, 385, 362, 373, 380]]

            # Calculate EAR for both eyes
            right_ear = calculate_ear(right_eye_landmarks)
            left_ear = calculate_ear(left_eye_landmarks)
            avg_ear = (right_ear + left_ear) / 2.0

            # Debugging: print EAR values to check behavior
            print(f"Right EAR: {right_ear:.2f}, Left EAR: {left_ear:.2f}, Avg EAR: {avg_ear:.2f}")
                        
            face_2d = np.array(face_2d, dtype=np.float64)
            face_3d = np.array(face_3d, dtype=np.float64)

            focal_length = 1 * img_w
            cam_matrix = np.array([[focal_length, 0, img_h / 2],
                                  [0, focal_length, img_w / 2],
                                  [0, 0, 1]])
            
            dist_matrix = np.zeros((4, 1), dtype=np.float64)
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)

            rmat, jac = cv2.Rodrigues(rot_vec)
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360

            # Determine head position
            if y < -10:
                text = "Looking Left"
                warning = "Warning: Please face forward!"
            elif y > 7:
                text = "Looking Right"
                warning = "Warning: Please face forward!"
            elif x < -10:
                text = "Looking Down"
                warning = "Warning: Please face forward!"
            elif x > 10:
                text = "Looking Up"
                warning = "Warning: Please face forward!"
            else:
                text = "Facing Forward"
                warning = ""
                
            # Check if eyes are closed
            if avg_ear < EAR_THRESHOLD and text == "Facing Forward":
                eyes_closed = True
                w = "Warning: Eyes are closed!"
                cv2.putText(image, w, (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 255), 3)

            # Draw line showing head pose direction
            nose_3d_projection, _ = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)
            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10))
            # cv2.line(image, p1, p2, (255, 0, 0), 3)

            # Display head position text
            cv2.putText(image, f"Status: {text}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (255, 0, 0), 3)
            if warning:
                cv2.putText(image, warning, (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 255), 3)
            
            # Display rotational angles
            cv2.putText(image, 'Pitch: ' + str(np.round(x, 2)), (1000, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, 'Yaw: ' + str(np.round(y, 2)), (1000, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, 'Roll: ' + str(np.round(z, 2)), (1000, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            end = time.time()
            fps = 1 / (end - start)
            cv2.putText(image, f'FPS: {int(fps)}', (20, 250), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)

        # Show the output image
        cv2.imshow("Head Pose Estimation", image)

        # Quit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('x'):
            break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Right EAR: 0.28, Left EAR: 0.46, Avg EAR: 0.37
Right EAR: 0.31, Left EAR: 0.48, Avg EAR: 0.39
Right EAR: 0.31, Left EAR: 0.47, Avg EAR: 0.39
Right EAR: 0.29, Left EAR: 0.47, Avg EAR: 0.38
Right EAR: 0.30, Left EAR: 0.47, Avg EAR: 0.38
Right EAR: 0.31, Left EAR: 0.48, Avg EAR: 0.39
Right EAR: 0.29, Left EAR: 0.46, Avg EAR: 0.37
Right EAR: 0.29, Left EAR: 0.47, Avg EAR: 0.38
Right EAR: 0.27, Left EAR: 0.45, Avg EAR: 0.36
Right EAR: 0.27, Left EAR: 0.45, Avg EAR: 0.36
Right EAR: 0.28, Left EAR: 0.47, Avg EAR: 0.38
Right EAR: 0.29, Left EAR: 0.48, Avg EAR: 0.39
Right EAR: 0.27, Left EAR: 0.47, Avg EAR: 0.37
Right EAR: 0.27, Left EAR: 0.47, Avg EAR: 0.37
Right EAR: 0.28, Left EAR: 0.46, Avg EAR: 0.37
Right EAR: 0.25, Left EAR: 0.46, Avg EAR: 0.36
Right EAR: 0.26, Left EAR: 0.46, Avg EAR: 0.36
Right EAR: 0.22, Left EAR: 0.43, Avg EAR: 0.32
Right EAR: 0.22, Left EAR: 0.43, Avg EAR: 0.33
Right EAR: 0.24, Left EAR: 0.45, Avg EAR: 0.34
Right EAR: 0.24, Left EAR: 0.44, Avg EAR: 0.34
Right EAR: 0.